# The role of drag on the global circulation an Aquaplanet
## Introduction
Until today it remains largely unknown what the role of drag (the extraction of momentum at the surface due to friction) is in the global circulation. As long as we do not understand this exactly, it is impossible to make projections on what the influence of large-scale land use change is on the climate. In order to understand the impact of land use change, we are going to look at a world without land. It might sound controversial, but it can help us a lot to understand the world better.

In this BSc-thesis project, you work with output of the ICON climate model, the new climate model of the Max Planck Institute for Meteorology (http://www.mpimet.mpg.de/en/science/models/icon/). The model has been run as an Aquaplanet in a setting with a diurnal cycle, but no seasons. The forcings of the model are symmetric over the equator, which means that the intertropical convergence zone (ITCZ) is centered exactly at the equator and the storm tracks on both hemispheres are equally far from the equator.

The drag at the surface is calculated following:

$$ \boldsymbol{\tau} = -C_D \left| \boldsymbol{u} \right| \boldsymbol{u},$$

which can be decomposed in the two vector components:

$$ \tau_u = -C_D \left| \boldsymbol{u} \right| u,$$
$$ \tau_v = -C_D \left| \boldsymbol{u} \right| v.$$

Let us go through these variables: $\tau$ is the drag at the surface in units of m$^2$ s$^{-2}$, vector $\boldsymbol{u} = (u,v)$ is the wind vector at the lowest model level consisting of the zonal component $u$ and the meridional component $v$, both in m s$^{-1}$. The expression $\left| \boldsymbol{u} \right|$ is the magnitude of the total wind vector, defined as $\left| \boldsymbol{u} \right| = \sqrt{u^2 + v^2}$.

The last variable is the drag coefficient $C_D$, which is the core of this project. The drag coefficient is a complex expression that under windy conditions can be approximated as:

$$C_D \approx \dfrac{\kappa^2}{\left( \ln \left( \dfrac{z}{z_{0m}} \right) \right)^2},$$

where $\kappa$ is the Von Karman constant with a value of 0.4, $z$ is the height at the first model level and $z_{0m}$ is the roughness length for momentum.

In this project, you are going to compare two simulations: one in which the roughness of water is prescribed as it is commonly done in weather and climate models, and one in which it is artificially enhanced with a factor of 100.

There are hypotheses that the position of the storm tracks, the locations of the strongest westerlies in the midlatitudes where most of the precipitation falls, is a function of the drag. It is your task to find out whether we observe this as well.

## Working plan
In this project, your goal is to understand the role of drag on the large-scale circulation better. This question can only be answered by answering a sequence of sub questions:

* What is difference in observed drag between the two simulations?
* What is the effect on the winds and on the climatology of wind?
* How do the global distribution of (geopotential) height, temperature, humidity and precipitation change?
* What is the explanation for the observed changes?

Answering all of these questions is, of course, unfeasible in the time you have for this project. Therefore, in the first week it is your task to explore the data, and search for differences that you consider important and that you find interesting. Based on those, you write your proposal and base the remainder of your thesis.


## The data
You work with a 3D NetCDF file that contains one year of 6-hourly fields of zonal wind $u$, meridional wind $v$, vertical wind $w$, height $zg$, temperature $T$, specific humidity $q$ and relative humidity $RH$. Each time step has the values at a lat/lon grid, with values at the 925, 850, 500 and 300 hPa pressure levels.

## Opening data with the help of Python
In order to be able to do our work with Python, it is necessary to load a number of packages and to change a few settings. Their purpose has been addded as comments in the code.

In [ ]:
from ipywidgets import interact, Dropdown, Select # Package ipywidgets contains the features of interactive notebooks.
import numpy as np    # Numpy is the fundamental package for scientific computing in Python.
import datetime as dt # Datetime is the basic package for turning a date string into a datetime object instance.
import netCDF4 as nc  # NetCDF has become the standard format for data storage in meteorology and climate sciences.
import matplotlib.pyplot as pl # Matplotlib is a scientific plotting package.
# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%pylab inline
pl.rcParams.update({'font.size': 12})          # Set the standard font size of the plots to 11pt.
pl.rcParams.update({'figure.figsize': [10,5]}) # Set the standard figure size.

First, we load the data set into an instance `nc_wind` of an object that can read a NetCDF file:

In [ ]:
nc_wind = nc.Dataset("u_p_3d.nc"      , "r") # The reference dataset is loaded in read-only mode.
#nc_wind = nc.Dataset("u_p_3d_small.nc", "r") # Small drag data set.
#nc_wind = nc.Dataset("u_p_3d_large.nc", "r") # Large drag data set.

The `nc_wind` instance gives us access to the variables from the data file. Note that we add the data range  `[:]` after the variable retrieval, to make sure we end up with an array of data points:

In [ ]:
lat  = nc_wind.variables["lat"] [:] # This is an array with the latitudes.
lon  = nc_wind.variables["lon"] [:] # This is an array with the longitudes.
lev  = nc_wind.variables["lev"] [:] # This is an array with the pressure levels.

# We process the time data into a datetime array that is the storage format for time data in Python.
time_tmp = nc_wind.variables["time"][:] # This is an array with the time variables.
time = []

for n in range(time_tmp.size):
    date_tmp, hour_tmp = divmod(time_tmp[n], 1)
    time.append(dt.datetime.strptime("{0}T{1}Z".format(int(date_tmp), int(hour_tmp*24)), "%Y%m%dT%HZ"))
    
time = np.array(time)


___
We can retrieve information about any type of variable by skipping the `[:]` and doing a print statement. It gives us the dimensions, the coordinates, a description and the units.

In [ ]:
print nc_wind.variables["ua"]

## Making some basic plots
We can take the latest slice of the zonal wind speed and plot it at the pressure level that is closest to the surface.

In [ ]:
ua = nc_wind.variables["ua"][-1,0,:,:]

In [ ]:
pl.pcolormesh(lon, lat, ua, cmap=pl.cm.magma)
pl.colorbar()
pl.xlim(lon.min(), lon.max())
pl.ylim(lat.min(), lat.max())
pl.xlabel("lon")
pl.ylabel("lat")
pl.title("u at {0} hPa (m/s) on {1}".format(lev[0]/100., time[-1]));

___
The interactive features of the notebook make it possible to create an interactive graph that is very helpful for data analysis. We have switched on manual update as the data can be large.

In [ ]:
def plot_u_on_p_surface(t = Dropdown(), p = Dropdown(), umin=-15, umax=25):
    ip = lev_list.index(p)
    it = time_list.index(t)
    ua = nc_wind.variables["ua"][it,ip,:,:]
    pl.pcolormesh(lon, lat, ua, vmin=umin, vmax=umax, cmap=pl.cm.magma)
    pl.colorbar()
    pl.xlim(lon.min(), lon.max())
    pl.ylim(lat.min(), lat.max())
    pl.xlabel("lon")
    pl.ylabel("lat")
    pl.title("u (m/s) at {0} hPa on {1}".format(lev[ip] / 100., time[it]));

time_list = [str(t) for t in time]
lev_list  = [str(l) for l in lev ]

time_box  = Select(description='Time'  , value=time_list[0], options=time_list)
level_box = Select(description='Levels', value=lev_list [0], options=lev_list )

interact(plot_u_on_p_surface, t=time_box, p=level_box, umin=(-100,0), umax=(0,100));

___
The scientific calculation package `numpy` provides many useful features for the analysis of the data. It can take averages of multidimensional data over any given axis. Below, we show how we construct a time mean from the total data, and subsequently the zonal time mean.

In [ ]:
ua_time_mean  = np.mean(nc_wind.variables["ua"][200:,-1,:,:], axis=0)
ua_zonal_mean = np.mean(nc_wind.variables["ua"][200:,-1,:,:], axis=2)
ua_time_zonal_std  = np.std (ua_zonal_mean, axis=0)
ua_time_zonal_mean = np.mean(ua_zonal_mean, axis=0)
pl.figure(figsize=(12,6))
pl.subplot(121)
pl.pcolormesh(lon, lat, ua_time_mean, cmap=pl.cm.plasma)
pl.colorbar()
pl.xlim(lon.min(), lon.max())
pl.ylim(lat.min(), lat.max())
pl.xlabel("lon")
pl.ylabel("lat")
pl.title("time mean u at {0} hPa (m/s)".format(lev[1] / 100.));
pl.subplot(122)
for n in range(200, time.size, 4):
    pl.plot(np.mean(nc_wind.variables["ua"][n,-1,:,:], 1), lat, color='#dddddd')
pl.plot(ua_time_zonal_mean - 2.*ua_time_zonal_std , lat, 'k:', label='95% range')
pl.plot(ua_time_zonal_mean + 2.*ua_time_zonal_std , lat, 'k:')
pl.plot(ua_time_zonal_mean, lat, label='mean(U)')
pl.ylim(lat.min(), lat.max())
pl.xlabel("u (m/s)")
pl.ylabel("lat")
pl.legend(frameon=False, loc=0) # Put the legend at the most suitable place.
pl.title("time and zonal mean u at {0} hPa (m/s)".format(lev[1] / 100.));

___
Below is shown an example for the plotting of a time series.

In [ ]:
print("Latitude: {0:.5}".format(lat[35]))        # Manually select the correct latitude
idx_lat40 = abs(lat-40).argmin()              # Argmin gives the index where the given array has the minimal value.
print("Latitude: {0:.5}".format(lat[idx_lat40])) # Check here whether the outcome is correct.
ua_time_40_850 = nc_wind.variables["ua"][:,1,idx_lat40,0]
print("Mean wind at 40 N, 0 E, and 850 hPa = {0:.4} m/s".format(ua_time_40_850.mean()))
print("Max  wind at 40 N, 0 E, and 850 hPa = {0:.4} m/s".format(ua_time_40_850.max()))
pl.plot(time, ua_time_40_850);
pl.xlabel('date');
pl.ylabel('u (m/s)');

## A few examples of advanced statistics
Here is an example of how to draw a PDF. We use all the data at 40 degrees latitude on 850 hPa.

In [ ]:
idx_lat45 = abs(lat-45).argmin()  
idx_lat00 = abs(lat- 0).argmin()

# We skip the first 200 time points to make sure that the model is properly spun up.
p45, up45 = np.histogram(nc_wind.variables["ua"][200:,1,idx_lat45,:], 50, density=True);
p00, up00 = np.histogram(nc_wind.variables["ua"][200:,1,idx_lat00,:], 50, density=True);

up00 = up00[:-1] + 0.5*(up00[1]-up00[0]) # Convert bin edges to centers.
up45 = up45[:-1] + 0.5*(up45[1]-up45[0]) # Convert bin edges to centers.

pl.plot(up00, p00, 'b-', label='0 N');
pl.plot(up45, p45, 'r-', label='45 N');
pl.xlabel('u (m/s)');
pl.ylabel('p');
pl.legend(frameon=False);

___
Here is a more advanced example. In this example we use Fourier transforms to find the wave number $k$ of the most relevant motions in the storm tracks. This technique is NOT assumed knowledge, it is only here to show you a technique how to answer such questions. This technique is called spectral analysis. The area under this curve is exactly equal to the variance of the data:

$$\sigma_u^2 = \int_0^{129} k\,E(k)\,d \ln(k)$$

In [ ]:
lat_jet = 40. # Choose the latitude at which we center the calculation.

idx_lat_nh = abs(lat-lat_jet).argmin() # The world is assumed to be symmetric. This means that we can get 
idx_lat_sh = abs(lat+lat_jet).argmin() # better statistics by taking both hemispheres.

ufft = np.zeros(129)
for n in range(200, time.size):
    ufft_tmp = np.fft.rfft(nc_wind.variables["ua"][n,1,idx_lat_nh,:]) / 256.
    ufft_tmp = abs(ufft_tmp)**2
    ufft_tmp[0] = 0.
    ufft_tmp[1:-1] *= 2
    
    ufft_tmp2 = np.fft.rfft(nc_wind.variables["ua"][n,1,idx_lat_sh,:]) / 256.
    ufft_tmp2 = abs(ufft_tmp2)**2
    ufft_tmp2[0] = 0.
    ufft_tmp2[1:-1] *= 2
    
    ufft += ufft_tmp
    ufft += ufft_tmp2
    
ufft /= ( 2.*(1461.-200.) )
k = arange(0,129,1)
pl.semilogx(k, k*ufft, 'bo-');
pl.xlabel('k');
pl.ylabel('k E(k)');
pl.grid()